In [136]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from PIL import Image

os.getcwd()

'/usr/users/vogg/FairMOT/own_ideas'

## Converting VoTT output

After labeling with VoTT, we obtain a .csv file, which we convert into 

In [2]:
data_root = "/usr/users/vogg/test/"

In [3]:
labels = pd.read_csv(data_root + "Validation-export.csv")

In [4]:
labels

,image,xmin,ymin,xmax,ymax,label
0,VID_20210223_123551.mp4#t=0.jpg,61.029412,10.690080,324.535604,360.552179,1
1,VID_20210223_123551.mp4#t=0.jpg,675.526316,662.877867,948.885449,999.061927,2
2,VID_20210223_123551.mp4#t=0.jpg,939.047988,774.368979,1139.520124,1075.262615,3
3,VID_20210223_123551.mp4#t=0.jpg,919.032508,628.144782,1068.119195,794.471904,4
4,VID_20210223_123551.mp4#t=0.jpg,1335.394737,204.613245,1559.976780,540.263188,5
...,...,...,...,...,...,...
908,VID_20210223_123551.mp4#t=10.333333.jpg,782.662539,642.055046,921.362229,943.266055,1
909,VID_20210223_123551.mp4#t=10.333333.jpg,671.702786,672.786124,899.566563,1070.091743,2
910,VID_20210223_123551.mp4#t=10.333333.jpg,1523.715170,430.018349,1719.876161,725.284404,3
911,VID_20210223_123551.mp4#t=10.333333.jpg,1278.018576,231.853211,1501.919505,529.100917,5


In [5]:
img_list = labels['image'].unique()

In [7]:
len(img_list)

164

In [8]:
img_id = img_list[0]

img = np.asarray(Image.open(data_root + "images/" + img_id))
#fig, ax = plt.subplots(figsize=(640/96, 640/96), dpi=96)
#ax.imshow(img)
img_h, img_w, _ = img.shape

In [9]:
img_w

1920

In [ ]:
for img_id in img_list:
    idx = np.where(labels.image == img_id)
    image_labels = labels.iloc[idx]
    
    for index, row in image_labels.iterrows():
        x_center = row.xmin + (row.xmax - row.xmin)/2
        y_center = row.ymin + (row.ymax - row.ymin)/2
        w = row.xmax - row.xmin
        h = row.ymax - row.ymin
        
        i_monkey = row.label

        img_id = re.sub("[.]jpg","", img_id)
        #Label-String schreiben.
        label_fpath = data_root + "labels_with_ids/" + img_id +'.txt'

        label_str = '0 {:d} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(
                i_monkey, x_center / img_w, y_center / img_h, w / img_w, h / img_h)

        with open(label_fpath, 'a') as f:
            f.write(label_str)

### Converting CVAT output

Data comes in separate files.

In [165]:
video_id = "VID_20210223_123854_0"

data_root = "/usr/users/agecker/datasets/macaque_videos/"

path_o = data_root + video_id + "/obj_train_data/"

frame_list = os.listdir(path_o)
frame_list.sort()

In [168]:
regex = re.compile('.*txt')
frame_list = [i for i in frame_list if regex.match(i)]
print(len(frame_list))

391


In [167]:
for frame_id in frame_list:
    text = pd.read_csv(path_o + frame_id, names = [1,2,3,4,5], sep = " ")
    text.insert(0, "z", np.zeros(len(text)))
    text.z = text.z.astype(int)
    text.to_csv(data_root + video_id + "/labels_with_ids/" + frame_id,
                 sep=' ', index=False, header = False)